In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [4]:
import pandas as pd

# Load dataset
df = pd.read_csv("dataset.csv")

# Combine date and time
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")

# Only keep required columns
df_clean = df[[
    'Timestamp', 'Occupancy', 'Capacity', 'QueueLength',
    'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType'
]]

# Save as CSV (overwrite any previous version)
df_clean.to_csv("parking_stream_clean.csv", index=False)

# Verify
print("✅ Columns in file:", df_clean.columns.tolist())


✅ Columns in file: ['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']


In [5]:
!head parking_stream_clean.csv


Timestamp,Occupancy,Capacity,QueueLength,TrafficConditionNearby,IsSpecialDay,VehicleType
2016-10-04 07:59:00,61,577,1,low,0,car
2016-10-04 08:25:00,64,577,1,low,0,car
2016-10-04 08:59:00,80,577,2,low,0,car
2016-10-04 09:32:00,107,577,2,low,0,car
2016-10-04 09:59:00,150,577,2,low,0,bike
2016-10-04 10:26:00,177,577,3,low,0,car
2016-10-04 10:59:00,219,577,6,high,0,truck
2016-10-04 11:25:00,247,577,5,average,0,car
2016-10-04 11:59:00,259,577,5,average,0,cycle


In [6]:
import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

# Use the NEW file you just saved
data = pw.demo.replay_csv("parking_stream_clean.csv", schema=ParkingSchema, input_rate=1000)


In [7]:
fmt = "%Y-%m-%d %H:%M:%S"
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


In [10]:
# ------------------------------
# Parameters and UDFs
# ------------------------------
BASE_PRICE = 10.0
LAMBDA = 0.5
MAX_PRICE = 2 * BASE_PRICE
MIN_PRICE = 0.5 * BASE_PRICE

# Traffic condition to numeric
@pw.udf
def traffic_level_mapper(traffic_str: str) -> int:
    return {"low": 0, "medium": 1, "high": 2}.get(traffic_str.lower(), 1)

# Vehicle type to weight
@pw.udf
def vehicle_weight(vehicle_type: str) -> float:
    return {"car": 1.0, "bike": 0.5, "truck": 2.0}.get(vehicle_type.lower(), 1.0)

# Demand function
@pw.udf
def compute_demand(occupancy: int, capacity: int, queue: int, traffic: int, is_special: int, vehicle_weight: float) -> float:
    if capacity == 0:
        return 0.0
    return (
        2.0 * (occupancy / capacity)
        + 1.5 * queue
        - 1.0 * traffic
        + 2.0 * is_special
        + 1.0 * vehicle_weight
    )

# Normalize demand and compute price
@pw.udf
def demand_to_price(demand: float) -> float:
    norm_demand = demand / 10  # Assuming demand is in a 0–10 range
    price = BASE_PRICE * (1 + LAMBDA * norm_demand)
    return max(min(price, MAX_PRICE), MIN_PRICE)

# ------------------------------
# Stage 1: Compute TrafficLevel and VehicleWeight
# ------------------------------
stage1 = data.with_columns(
    TrafficLevel=traffic_level_mapper(data.TrafficConditionNearby),
    VehicleWeight=vehicle_weight(data.VehicleType)
)

# ------------------------------
# Stage 2: Compute Demand
# ------------------------------
stage2 = stage1.with_columns(
    Demand=compute_demand(
        stage1.Occupancy,
        stage1.Capacity,
        stage1.QueueLength,
        stage1.TrafficLevel,
        stage1.IsSpecialDay,
        stage1.VehicleWeight
    )
)

# ------------------------------
# Stage 3: Compute Price and final output
# ------------------------------
pricing_table = stage2.with_columns(
    Price=demand_to_price(stage2.Demand),
    Timestamp=stage2.Timestamp,
    t=stage2.t,
    day=stage2.day
)

# ------------------------------
# Save Output for Visualization
# ------------------------------
pw.io.jsonlines.write(pricing_table, "/tmp/model2_output.jsonl")


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [11]:
pw.run()

Output()

In [12]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Show Bokeh plots in notebook
output_notebook()

# Load Model 2 output
df = pd.read_json("/tmp/model2_output.jsonl", lines=True)

# Ensure proper datetime type
df['t'] = pd.to_datetime(df['t'])

# Sort for clean plotting
df = df.sort_values('t')

# Prepare Bokeh source
source = ColumnDataSource(df)

# Create figure
fig = figure(
    title="Dynamic Parking Price Over Time (Model 2)",
    x_axis_type="datetime",
    width=800,
    height=400
)

# Add line + points
fig.line(x='t', y='Price', source=source, line_width=2, color='green', legend_label="Model 2 Price")
fig.circle(x='t', y='Price', source=source, size=5, color='darkgreen')

# Axis labels
fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.legend.location = "top_left"

# Show plot
show(fig)


Output hidden; open in https://colab.research.google.com to view.